In [3]:
import simpy

In [4]:
!python -V

Python 3.5.2 :: Anaconda custom (x86_64)


In [5]:
>>> class Car(object):
...     def __init__(self, env):
...         self.env = env
...         self.action = env.process(self.run())
...
...     def run(self):
...         while True:
...             print('Start parking and charging at %d' % self.env.now)
...             charge_duration = 5
...             # We may get interrupted while charging the battery
...             try:
...                 yield self.env.process(self.charge(charge_duration))
...             except simpy.Interrupt:
...                 # When we received an interrupt, we stop charging and
...                 # switch to the "driving" state
...                 print('Was interrupted. Hope, the battery is full enough ...')
...
...             print('Start driving at %d' % self.env.now)
...             trip_duration = 2
...             yield self.env.timeout(trip_duration)
...
...     def charge(self, duration):
...         yield self.env.timeout(duration)

>>> def driver(env, car):
...     yield env.timeout(3)
...     car.action.interrupt()


>>> def car(env, name, bcs, driving_time, charge_duration):
        # Simulate driving to the BCS
        yield env.timeout(driving_time)
        # Request one of its charging spots
        print('%s arriving at %d' % (name, env.now))
        number_of_cars_charging = 0
        with bcs.request() as req:
            yield req
            
            # Charge the battery
            print('%s starting to charge at %s' % (name, env.now))
            number_of_cars_charging+=1
            print()
            print('There are {} cars charging start'.format(number_of_cars_charging))
            yield env.timeout(charge_duration)
            print('%s leaving the bcs at %s' % (name, env.now))
            number_of_cars_charging -=1
            print('There are {} cars charging left'.format(number_of_cars_charging))



In [6]:
>>> env = simpy.Environment()
>>> bcs = simpy.Resource(env, capacity=2)

for i in range(4):
    print(i)
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

0
1
2
3


In [7]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0

There are 1 cars charging start
Car 1 arriving at 2
Car 1 starting to charge at 2

There are 1 cars charging start
Car 2 arriving at 4
Car 0 leaving the bcs at 5
There are 0 cars charging left
Car 2 starting to charge at 5

There are 1 cars charging start
Car 3 arriving at 6
Car 1 leaving the bcs at 7
There are 0 cars charging left
Car 3 starting to charge at 7

There are 1 cars charging start
Car 2 leaving the bcs at 10
There are 0 cars charging left
Car 3 leaving the bcs at 12
There are 0 cars charging left


In [8]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random

import simpy




def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    global total_card_holders ### should set this as a class variable in the future
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        
        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            total_card_holders += counter.count
            
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))
            ### assign people who reach the counter a credit card for 100 time steps
            number_of_card_holders = simpy.Resource(env, capacity=1000)
            yield env.process(purchase_card(env,name,1,number_of_card_holders))
            #purchase_card(env,name,1)
            
        
        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

    
    
def purchase_card(env, n, number,num_card_holders):
    """Generate 'card purchases' for people who reached the bank teller"""
    global total_card_holders
    purchase_card_time = env.now
    with num_card_holders.request() as req:
        
    #card_purchaser = ('Card-Holder%02d' %  total_card_holders, card_lifetime=100.0)
  
        yield env.timeout(1) # no delay
        print( " {} got a credit card at time {}".format(n,purchase_card_time,purchase_card_time))
        



In [11]:
# Setup and start the simulation
print('Bank renege')
random.seed(10)
env = simpy.Environment()



RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience

# Start processes and run
counter = simpy.Resource(env, capacity=1)
total_card_holders = 0
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()
print('Total card holdres finished {}'.format(total_card_holders ))

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 8.4724 Customer01: Here I am
10.3556 Customer00: Finished
10.7803 Customer02: Here I am
11.0990 Customer01: RENEGED after  2.627
 Customer00 got a credit card at time 10.355596101037406
11.3556 Customer02: Waited  0.575
13.4511 Customer02: Finished
 Customer02 got a credit card at time 13.451116573793573
28.1297 Customer03: Here I am
28.1297 Customer03: Waited  0.000
31.5818 Customer03: Finished
 Customer03 got a credit card at time 31.58184019457356
35.4834 Customer04: Here I am
35.4834 Customer04: Waited  0.000
36.0303 Customer04: Finished
 Customer04 got a credit card at time 36.03030927790481
66.0206 Customer05: Here I am
66.0206 Customer05: Waited  0.000
71.7881 Customer05: Finished
 Customer05 got a credit card at time 71.78810546279168
85.6932 Customer06: Here I am
85.6932 Customer06: Waited  0.000
89.0286 Customer07: Here I am
91.3523 Customer07: RENEGED after  2.324
93.0172 Customer06: Finished
 Custo

In [10]:
>>> def sub(env):
...     yield env.timeout(1)
...     return 23
...
>>> def parent(env):
...     ret = yield env.process(sub(env))
...     return ret
...
>>> env.run(env.process(parent(env)))


23

In [18]:
import numpy as np

In [21]:
def customer(env,number_of_weekly_customers, avg_customer_lifetime_weeks):
    """This creates a simpy process for each customer where the lifetime
    of each customer is defined by an exponenetial distribution."""
    print("Starting customer transaction at {}".format(env.now))

    customer_lifetime = np.random.exponential(scale =
                                              1/avg_customer_lifetime_weeks)
    # start the process for each customer's lifetime

    for i in range(number_of_weekly_customers): # one customer for the ones we
        #received this  week
        c = open_bank_account(env, 'Customer%02d' % i,
                              cust_lifetime = avg_customer_lifetime_weeks)
        env.process(c)
        # how frequently do coustomers come in?
        time_between_customers = \
            random.expovariate(1.0 / number_of_weekly_customers)
        print('The time between each customer is {} weeks'.format(
            time_between_customers))
        yield env.timeout(time_between_customers) # time in weeks between customers


def open_bank_account(env,customer_id,cust_lifetime):
    """This is a simpy process for creating a bank account"""
    global total_customer_with_bankaccount  # set to class var in future
    print(' {} opened a bank account'.format(customer_id))
    total_customer_with_bankaccount +=1
    print(' There are {} people with bank accounts at time-weeks {}'.format(
        total_customer_with_bankaccount, env.now
    ))
    yield env.timeout(cust_lifetime)


In [22]:
env = simpy.Environment()
env.process(customer(env,10,100))
env.run(until=100)

Starting customer transaction at 0
The time between each customer is 0.45270031620124374 weeks
 Customer00 opened a bank account


NameError: name 'total_customer_with_bankaccount' is not defined

In [27]:
[np.random.exponential(100) for _ in range(100)]

[30.733270941562385,
 9.912318158116713,
 277.0957472588501,
 18.558318820651255,
 24.29235546794653,
 33.797585213965256,
 41.272001949311665,
 369.8779388987962,
 97.29785689775596,
 195.70654972061078,
 121.58428575160742,
 252.08624956207726,
 39.0333600803136,
 79.10382109540734,
 123.52503420819112,
 83.98223803175104,
 332.6650683369942,
 107.98442420701926,
 40.22612819543268,
 211.90652876629787,
 151.36519633217057,
 30.41399714910088,
 39.333728468191225,
 280.26037354437176,
 33.28870808391596,
 178.30942815458272,
 20.380793142626086,
 36.00762142752567,
 303.8812337528769,
 34.66694324106035,
 27.033363151782265,
 0.4371399924492804,
 90.38401827298297,
 80.99710339181767,
 407.8096960120597,
 9.180392352167841,
 7.366699666850504,
 75.4907738824404,
 244.73051125038813,
 67.28254197980795,
 1.5026637845425623,
 22.116839690887264,
 130.13037358297152,
 27.396881576583876,
 78.65229020642181,
 264.7931088246426,
 278.1376291369951,
 34.013622968735,
 65.50312833833333,
 4